In [1]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional, Callable, Union
from typing  import Literal
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ItemHelpers,
    TResponseInputItem,
    function_tool,
    FunctionTool,
    handoff,
    ModelSettings
)
load_dotenv()

from helpers.trace_util import get_trace_url
from helpers.vis_workflow import vis_agent
import json
import random

In [2]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)


In [3]:

class StockData(BaseModel):
    """Model for stock data"""
    price: float = Field(..., description="Current stock price")
    volume: int = Field(..., description="Trading volume")
    pe_ratio: float = Field(..., description="Price-to-earnings ratio")
    market_cap: str = Field(..., description="Market capitalization")

@function_tool
def get_stock_data(symbol: str) -> StockData:
    """Get stock market data for a given symbol"""
    return StockData(**{"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"})

class NewsItem(BaseModel):
    """Model for news items"""
    title: str = Field(..., description="Title of the news article")
    date: str = Field(..., description="Publication date")
    summary: str = Field(..., description="Summary of the article")

@function_tool
def get_news(query: str) -> List[NewsItem]:
    """Get recent news articles about a company"""
    return [
        NewsItem(**
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Q1 Earnings Report",
            "date": "2024-03-21",
            "summary": "Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.",
        }),
        NewsItem(**
        {
            "title": "Tesla Partners with Panasonic",
            "date": "2024-03-22",
            "summary": "Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.",
        }),
        NewsItem(**
        {
            "title": "Tesla Model S Plaid Review",
            "date": "2024-03-23",
            "summary": "The Tesla Model S Plaid receives rave reviews for its performance and technology features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Energy Division Grows",
            "date": "2024-03-24",
            "summary": "Tesla's energy division sees significant growth in solar and battery storage solutions.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Global Expansion Plans",
            "date": "2024-03-25",
            "summary": "Tesla announces plans to expand its manufacturing footprint in Europe and Asia.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Autopilot Update",
            "date": "2024-03-26",
            "summary": "Tesla releases a new update for its Autopilot system, improving safety and navigation features.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Stock Performance",
            "date": "2024-03-27",
            "summary": "Tesla's stock shows strong performance in the market, driven by positive investor sentiment.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Sustainability Initiatives",
            "date": "2024-03-28",
            "summary": "Tesla announces new sustainability initiatives aimed at reducing carbon footprint.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Cybertruck Launch Date Announced",
            "date": "2024-03-29",
            "summary": "Tesla announces the official launch date for the highly anticipated Cybertruck.",
        }),
        NewsItem(**
        {
            "title": "Tesla's AI Day 2024 Highlights",
            "date": "2024-03-30",
            "summary": "Tesla's AI Day showcases advancements in artificial intelligence and robotics.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Supercharger Network Expansion",
            "date": "2024-03-31",
            "summary": "Tesla announces plans to expand its Supercharger network across North America.",
        }),
        NewsItem(**
        {
            "title": "Tesla's Model 3 Production Update",
            "date": "2024-03-17",
            "summary": "Tesla increases production capacity for the Model 3 to meet growing demand.",
        }),
        NewsItem(**
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        }),
        NewsItem(**
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        }),
    ]



In [4]:
stockdata = await get_stock_data.on_invoke_tool(None, json.dumps({"symbol": "TSLA"}))
print(stockdata)
news = await get_news.on_invoke_tool(None, json.dumps({"query": "Tesla"}))
print(news)


price=180.25 volume=1000000 pe_ratio=65.4 market_cap='700B'
[NewsItem(title='Tesla Expands Cybertruck Production', date='2024-03-20', summary='Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.'), NewsItem(title="Tesla's Q1 Earnings Report", date='2024-03-21', summary='Analysts predict strong earnings for Tesla in the upcoming quarterly report, driven by record deliveries.'), NewsItem(title='Tesla Partners with Panasonic', date='2024-03-22', summary='Tesla and Panasonic announce a new partnership to enhance battery technology for electric vehicles.'), NewsItem(title='Tesla Model S Plaid Review', date='2024-03-23', summary='The Tesla Model S Plaid receives rave reviews for its performance and technology features.'), NewsItem(title="Tesla's Energy Division Grows", date='2024-03-24', summary="Tesla's energy division sees significant growth in solar and battery storage solutions."), NewsItem(title="Tesla's Global Expansion Plans", date='202

In [ ]:
financial_analyst = Agent(
    name="financial_analyst",
    model=model_name,
    tools=[get_stock_data],
    output_type=StockData,
    instructions="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    ### IMPORTANT ###
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a financial analyst.
    You perform analysis of the stock market data.
    you must handoff back to the planner agent."""
)


news_analyst = Agent(
    name="news_analyst",
    model=model_name,
    tools=[get_news],
    output_type=List[NewsItem],
    instructions="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
    handoff_description="""You are a news analyst.
    You retrieve news articles to analyze.
    you must handoff back to the planner agent.""",

)


writer = Agent(
    name="writer",
    model=model_name,
    tools=[],
    output_type=str,
    instructions="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    The report should be easy to understand for a general audience.
    The report is formatted as a markdown document.
    Always handoff back to planner when writing is complete.""",
    handoff_description="""You are a writer.
    You summary a summary of the analysis results.
    you must handoff back to the planner agent."""
)


try:
    vis_agent(financial_analyst, "11_planner_financial_analyst")
    vis_agent(news_analyst, "11_planner_news_analyst")
    vis_agent(writer, "11_planner_writer")
except Exception as e:
    print(f"An error occurred: {e}")
    pass

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [6]:
msg = "what is the TSLA stock, write a report"
input_items : list[TResponseInputItem] = [{"content" : msg, "role": "user"}]

In [7]:
planner_assistant = Agent(
    name="planner",
    model=model_name,
    handoffs=[financial_analyst,news_analyst, writer],
    instructions="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
    handoff_description="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents
    you are responsible for generating the final output.,
    """,

)

planner_assistant.handoffs = [writer]
planner_assistant.tools = [
    financial_analyst.as_tool(
        tool_name="financial_analyst",
        tool_description="Analyze stock market data using the get_stock_data tool. Provide insights on financial metrics."
    ),
    news_analyst.as_tool(
        tool_name="news_analyst",
        tool_description="Gather and analyze relevant news using the get_news tool. Summarize key market insights from news."
    ),

]
# financial_analyst.handoffs = [handoff(planner_assistant)]
# news_analyst.handoffs = [handoff(planner_assistant)]
# writer.handoffs = [handoff(planner_assistant)]

try:
    vis_agent(planner_assistant, filename="11_planner_financial_planner_as_tools")
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: vis_agent() got an unexpected keyword argument 'filename'


In [8]:
with trace(workflow_name="financial_planner", group_id="financial_planner") as tr:
    print(f"Tracing url: {get_trace_url(tr)}")

    result = await Runner.run(
        planner_assistant,
        input=input_items,
        run_config=RunConfig(
            model=model_name,
            model_provider=GITHUB_MODEL_PROVIDER,
            model_settings=ModelSettings(**{
                # "temperature": 0.5,
                "max_tokens": 5000,
                # "top_p": 1.0,
                # "frequency_penalty": 0.0,
                # "presence_penalty": 0.0,
                # "stop": ["\n\n"]
                }, # Adjust the model settings as needed
        )
        )


    )

    print(result)

Tracing url: https://platform.openai.com/traces/trace?trace_id=trace_029c480f183d4bc4aef7fd9e38efdc44
RunResult:
- Last agent: Agent(name="writer", ...)
- Final output (str):
    # Tesla, Inc. (TSLA) Stock Report
    
    ## Executive Summary
    Tesla, Inc. (TSLA) continues to be a leader in the electric vehicle market, showcasing robust financials and significant company developments as of March 2024. This report summarizes the current stock performance, financial metrics, and recent news developments that indicate the company's continued growth and innovation.
    
    ## Financial Analysis
    - **Current Price**: $180.25
    - **Trading Volume**: 1,000,000 shares
    - **Price-to-Earnings Ratio (P/E)**: 65.4
    - **Market Capitalization**: $700 billion
    
    These metrics indicate that Tesla's stock remains highly valued, suggesting investor confidence despite a relatively high P/E ratio.
    
    ## Recent Developments and News
    1. **Production Expansion**: Tesla has rampe

In [11]:
result.raw_responses[-1].usage

Usage(requests=1, input_tokens=1649, output_tokens=756, total_tokens=2405)

In [12]:
display(Markdown(result.final_output))

# Tesla, Inc. (TSLA) Stock Report

## Executive Summary
Tesla, Inc. (TSLA) continues to be a leader in the electric vehicle market, showcasing robust financials and significant company developments as of March 2024. This report summarizes the current stock performance, financial metrics, and recent news developments that indicate the company's continued growth and innovation.

## Financial Analysis
- **Current Price**: $180.25
- **Trading Volume**: 1,000,000 shares
- **Price-to-Earnings Ratio (P/E)**: 65.4
- **Market Capitalization**: $700 billion

These metrics indicate that Tesla's stock remains highly valued, suggesting investor confidence despite a relatively high P/E ratio.

## Recent Developments and News
1. **Production Expansion**: Tesla has ramped up production capacity for its Cybertruck at Gigafactory Texas to meet growing demand (March 20, 2024).
2. **Strong Financial Outlook**: Analysts predict strong earnings in the upcoming quarterly report, supported by record vehicle deliveries (March 21, 2024).
3. **Strategic Partnerships**: Tesla has announced a partnership with Panasonic to enhance battery technology, essential in staying competitive in the EV market (March 22, 2024).
4. **Model S Plaid Success**: The Model S Plaid has been well-received in the market, praised for its advanced performance (March 23, 2024).
5. **Energy Division Growth**: The company's energy division is experiencing significant growth, particularly in solar energy and battery storage solutions (March 24, 2024).
6. **Global Expansion**: Plans for expanding Tesla's manufacturing footprint in Europe and Asia have been announced (March 25, 2024).
7. **Technology Improvements**: Updates to Tesla's Autopilot system have been released, resulting in improved safety and navigation features (March 26, 2024).
8. **Stock Performance**: Overall, TSLA shares have shown strong performance amid positive investor sentiment (March 27, 2024).
9. **Sustainability Initiatives**: New initiatives have been shared to promote sustainability and reduce the company's carbon footprint (March 28, 2024).
10. **Anticipated Cybertruck Launch**: An official launch date for the Cybertruck has been announced (March 29, 2024).
11. **AI Innovations**: During AI Day 2024, Tesla showcased advancements in artificial intelligence relevant to its vehicles (March 30, 2024).
12. **Infrastructure Improvements**: Tesla's Supercharger network is set to expand significantly across North America (March 31, 2024).
13. **Increased Model 3 Production**: An increase in production capacity for the Model 3 was reported in response to rising demand (March 17, 2024).
14. **Full Self-Driving Updates**: Latest updates to the Full Self-Driving (FSD) beta show considerable promise in urban navigation (March 19, 2024).
15. **Best-Selling Model**: The Model Y has become the best-selling electric vehicle worldwide, solidifying its market position (March 18, 2024).

## Conclusion
Tesla, Inc. shows promising growth and a strong market position through strategic initiatives, technological advancements, and a solid financial foundation. Investors and stakeholders should remain attentive to the upcoming earnings report and the company's continued developments in the EV space.

This report highlights Tesla's resilience and potential for future growth as it continues to innovate and expand its market presence.

---

Should you require any further information or analysis, please feel free to reach out.